# Read in Data

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('yelp.csv')

In [2]:
df['stars'].value_counts()

4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64

In [3]:
df['labels'] = [0 if i in [1,2] else -1 if i == 3 else 1 for i in df['stars']]

In [4]:
mask = [False if i == -1 else True for i in df['labels']]

# Train Test Split

In [5]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[mask]['text'], df[mask]['labels'])

In [7]:
print 1.0*y_train.sum()/len(y_train)
print 1.0*y_test.sum()/len(y_test)

0.803716427233
0.8037470726


# Features

In [12]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import numpy as np

In [13]:
vect = CountVectorizer().fit(np.array(X_train))
train = vect.transform(np.array(X_train))
test =  vect.transform(np.array(X_test))

In [15]:
svd = TruncatedSVD(n_components=20, random_state=42)
svd.fit(train) 

TruncatedSVD(algorithm='randomized', n_components=20, n_iter=5,
       random_state=42, tol=0.0)

In [16]:
train_svd = svd.transform(train)
test_svd = svd.transform(test)

# Model

In [21]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

In [38]:
#### Naive Bayes
mnb = MultinomialNB()
mnb.fit(train, y_train)
mnb_pred = mnb.predict(test)

print 1.0*sum(mnb_pred == y_test)/len(y_test)
confusion_matrix(y_test, mnb_pred)

0.884309133489


array([[ 232,  187],
       [  60, 1656]])

In [39]:
#### Logistic Regression 
lr = LogisticRegression()
lr.fit(train, y_train)
lr_pred = mnb.predict(test)

print 1.0*sum(lr_pred == y_test)/len(y_test)
confusion_matrix(y_test, lr_pred)

0.884309133489


array([[ 232,  187],
       [  60, 1656]])

In [37]:
### SVD
log_svd = LogisticRegression()
log_svd.fit(train_svd, y_train)
log_svd_pred = log_svd.predict(test_svd)

print 1.0*sum(log_svd_pred == y_test)/len(y_test)
confusion_matrix(y_test, log_svd_pred)


0.826229508197


array([[  90,  329],
       [  42, 1674]])

In [40]:
svd_200 = TruncatedSVD(n_components=200, random_state=42)
svd_200.fit(train) 

TruncatedSVD(algorithm='randomized', n_components=200, n_iter=5,
       random_state=42, tol=0.0)

In [41]:
train_svd_200 = svd_200.transform(train)
test_svd_200 = svd_200.transform(test)

In [43]:
log_svd_200 = LogisticRegression()
log_svd_200.fit(train_svd_200, y_train)
log_svd_200_pred = log_svd_200.predict(test_svd_200)

print 1.0*sum(log_svd_200_pred == y_test)/len(y_test)
confusion_matrix(y_test, log_svd_200_pred)

0.885714285714


array([[ 242,  177],
       [  67, 1649]])

In [44]:
#stop words min_df, bigrams
vect2 = CountVectorizer(stop_words='english', ngram_range=(1,2) ,min_df=3).fit(np.array(X_train))
train2 = vect2.transform(np.array(X_train))
test2 =  vect2.transform(np.array(X_test))

In [45]:
svd_200 = TruncatedSVD(n_components=250, random_state=42)
svd_200.fit(train2) 

TruncatedSVD(algorithm='randomized', n_components=250, n_iter=5,
       random_state=42, tol=0.0)

In [46]:
train_svd_200 = svd_200.transform(train2)
test_svd_200 = svd_200.transform(test2)

In [47]:
log_svd_200 = LogisticRegression()
log_svd_200.fit(train_svd_200, y_train)
log_svd_200_pred = log_svd_200.predict(test_svd_200)

print 1.0*sum(log_svd_200_pred == y_test)/len(y_test)
confusion_matrix(y_test, log_svd_200_pred)

0.88149882904


array([[ 242,  177],
       [  76, 1640]])

# Game


**Analytical Solution**
For a board of length M, we need to move forward M times. And the rest of the moves are stays. We will have N-M of those.

So we can view this as a combination. Of the N moves, we have to pick M to the forwards. So that's N-Combination-M, which is **N!/(M!(N-M)!)**

It can also be viewed as a permutation of N moves, where M are of type F and (N-M) of stays. So it's permutations with repetitions: N!/(M!(N-M)!). Which is the same thing as before.

In [48]:
import numpy as np
def fac(n):
    return np.prod(range(1,n+1))

In [49]:
def ways(n,m):
    return fac(n)/(fac(n-m)*fac(m))

In [50]:
ways(8,5)

56

### Recursion


For recursion, we just have to figure out the first step. In this case, we can either stay or go. If we stay, we have N-1 moves and M positions. If we go forward, we have N-1 moves and M-1 postions. The total solutions is the sum of the two options. So we'll just sum them up.
For every recursion problem, the steps we take are the same 3.
* What is the first step (in this case: go forward or stay)
* How do we combine the pieces (in this case: sum the two cases)
* What are the end conditions (in this case: if M>N then 0, if M=1 then N)

If M=1 then N or If M=N then 1.

In [51]:
def ways2(n,m):
    if n<m or m<0:
        return 0
    elif n==m:
        return 1
    else:
        return ways2(n-1,m)+ways2(n-1,m-1)

In [53]:
ways2(8,5)

56